In [1]:
# Install the datasets package
!pip3 install datasets

from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Inspect the dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})


In [2]:
train_test_split = dataset["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
valid_dataset = train_test_split["test"]

# Print the sizes of the splits
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(valid_dataset)}")


Training set size: 4505
Validation set size: 501
